In [ ]:
from globalConstants import *
import dynamiqs as dq
import jax
import jax.numpy as jnp # the JAX version of numpy
from matplotlib import pyplot as plt


In [ ]:
import dynamiqs as dq
import jax.numpy as jnp

# Corrected Parameters with Units
GHz = 1e9
MHz = 1e6
h = 6.62607015e-34  

# Frequencies (converted to rad/s)
wa0 = 5.26 * GHz * 2 * jnp.pi  # 5.26 GHz to rad/s
wb0 = 7.7 * GHz * 2 * jnp.pi
omegaP = 2.891 * GHz * 2 * jnp.pi
wD = 7.623 * GHz * 2 * jnp.pi

# E_J and ΔE_J in rad/s (E_J / ħ)
E_J_Hz = 42.76 * GHz
E_J_rads = E_J_Hz * 2 * jnp.pi
delEJ_Hz = 0.47 * GHz
delEJ_rads = delEJ_Hz * 2 * jnp.pi

# Drive and pump parameters
epsD = -3.815 * MHz * 2 * jnp.pi  # ε_d in rad/s
epsP = 0.122  # radians (dimensionless)

# Dissipation rates (from paper: κ_a/2π=0.05 MHz, κ_b/2π=5 MHz)
kappa_a = 0.05 * MHz * 2 * jnp.pi  # rad/s
kappa_b = 5 * MHz * 2 * jnp.pi      # rad/s

# Hilbert space truncation
n_a, n_b = 20, 5




# Phi operator
phi_a_val = 0.06
phi_b_val = 0.29
phi = phi_a_val * (a_tensored + adag_tensored) + phi_b_val * (b_tensored + bdag_tensored)

# Jump operators
jump_ops = [
    jnp.sqrt(kappa_b) * b_tensored,
    jnp.sqrt(kappa_a) * a_tensored
]

# Initial state: vacuum
psi0 = dq.tensor(dq.fock(n_a, 0), dq.fock(n_b, 0))

# Time parameters
T = 1e-6  # 1 μs (adjust as needed)
num_steps = 1000
t_save = jnp.linspace(0, T, num_steps)

# Hamiltonian function with corrected expansions
def H_func(t):
    eps_t = epsP * jnp.cos(omegaP * t)
    
    Hd = 2 * epsD * jnp.cos(wD * t) * (b_tensored + bdag_tensored)
    
    # Corrected expansions for sin(φ) and cos(φ)
    # Replace divisions in operator expressions with multiplication by reciprocals
    

    # Corrected H_ATS construction
    H_ATS = (
        -2 * E_J_rads * jnp.sin(eps_t) * sin_phi_approx
        + 2 * delEJ_rads * jnp.cos(eps_t) * cos_phi_approx
    )

# Ensure all operator terms use * for scalar multiplication
    
    return H0 + Hd + H_ATS

# Run the simulation
result = dq.mesolve(
    dq.timecallable(H_func),
    jump_ops,
    psi0,
    t_save,
)

# Proceed with plotting and analysis